In [3]:
pip install SPARQLWrapper

Note: you may need to restart the kernel to use updated packages.


In [3]:
import random
from SPARQLWrapper import SPARQLWrapper, JSON

In [4]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql",agent="Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11")

In [171]:
def getQidLabel(label):
    sparql.setQuery("""
    SELECT ?item WHERE {
      ?item rdfs:label \""""+label+"""\"@en
    }
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    result =results["results"]["bindings"][0]["item"]["value"]
    quid = result.split('/')
    return quid[len(quid)-1]
    

In [172]:
getQidLabel('Albania')

'Q222'

In [176]:
def makelabeldict(Qid):
    sparql.setQuery("""
    select distinct ?subject ?subjectLabel ?property ?propertyLabel {
      values (?item) {(wd:"""+Qid+""")}
      ?subject ?predicate ?item .
      ?property wikibase:directClaim ?predicate
      service wikibase:label { bd:serviceParam wikibase:language "en" }
    }Limit 90
    """)
    #/wdt:P31/wdt:P279
    
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    mydict = []
    for result in results["results"]["bindings"]:
        mydict.append((result["propertyLabel"]["value"], result["subjectLabel"]["value"]))
    return mydict

In [177]:
makelabeldict(getQidLabel('Albania'))

[('country', 'Sinaballaj'),
 ('country', 'Drisht Castle'),
 ('country', 'Borsh'),
 ('country', 'ALBtelecom'),
 ('country', 'Environmentalist Agrarian Party'),
 ('country', 'Amantia'),
 ('country', 'Albanian Football Association'),
 ('country', 'Bojana'),
 ('country', 'Šar Mountains'),
 ('country', 'Albanian Declaration of Independence'),
 ('country', 'Skelë Castle'),
 ('country', 'Barbas, Albania'),
 ('country', 'Skanderbeg Monument'),
 ('country', 'Jablanica Mountain'),
 ('country', 'Myzeqe'),
 ('country', 'Trans Adriatic Pipeline'),
 ('country', 'Roman Catholic Archdiocese of Tiranë-Durrës'),
 ('country', '1972–73 Albanian Superliga'),
 ('country', '1993–94 Albanian Superliga'),
 ('country', 'Fushë-Arrëz'),
 ('country', 'Balkan Cup'),
 ('country', 'Albania at the 2011 World Aquatics Championships'),
 ('country', 'Zagori'),
 ('country', 'Q615228'),
 ('country', 'Burrel'),
 ('country', '2004–05 Albanian Cup'),
 ('country', 'Gjirokastër National Folklore Festival'),
 ('country', '1966–6

In [57]:
def research(Qid):
    sparql.setQuery("""
    SELECT ?item ?itemLabel
    WHERE 
    {
      ?item wdt:P31 wd:""" + Qid + """. #on pourais rajouté des obligation de référance à des payes etc
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } # le label viendra de préférence dans votre langue, et autrement en anglais
      
    }LIMIT 1000
    """)
    #/wdt:P31/wdt:P279
    
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    mydict = []
    for result in results["results"]["bindings"]:
        # ne prend pas ce ou il y a pas de nom ajouté
        if(result["itemLabel"]["value"][0]!="Q"):
            mydict.append(result["itemLabel"]["value"])
            
    return mydict

In [58]:
listofidqHuman = ['Q22808320', 'Q571', 'Q31629', 'Q188451', 'Q192581', 'Q11424', 'Q11410', 'Q2095', 'Q3624078']


In [59]:
description=[]
for i in range(len(listofidqHuman)):
    description.append(research(listofidqHuman[i]))

head
results
head
results
head
results
head
results
head
results
head
results
head
results
head
results
head
results


In [32]:
for descript in description:
    print(descript)

['David Terry', 'Robert Newton', 'William Meredith', 'Robert Rose', 'William Gibson', 'Richard Thompson', 'John Gillis', 'William Hamilton', 'Ian Hamilton', 'José Martínez', 'James Johnson', 'Alan Ball', 'Thrasydaios', 'Mikhail Biryukov', 'Mark Foster', 'Arthur Mitchell', 'James Rogers', 'George Curzon', 'Francisco Hernández de Córdoba', 'Charles V', 'Johann Strauss', 'Michael Howard', 'Keith Campbell', 'Lukas Bauer', 'David McNally', 'William Jones', 'Gareth Thomas', 'Henry II', 'Jesse Williams', 'Michael Anderson', 'George Bush', 'Chris Nelson', 'John Carroll', "Anna de' Medici", 'Henk Visser', 'Nikos Politis', 'Alan Clark', 'Peter Karageorgevitch', 'Mikko Lehtonen', 'Manuel I', 'Manuel II', 'Margaret of Burgundy', 'Meto Jovanovski', 'Norman Scott', 'Hatice Sultan', 'Michael Clarke', 'Alex Harvey', 'Alexander Campbell', 'Alexander Gray', 'Alexander Smirnov', 'Aleksandr Petrov', 'Alexei Petrov', 'Walter Forster', 'John Chambers', 'Peter Schneider', 'Franz Koch', 'James Dennis', 'John 

In [8]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration,Adafactor

In [9]:
model_path = "./generation model/"

In [10]:
tokenizer = T5Tokenizer.from_pretrained(model_path)
model =T5ForConditionalGeneration.from_pretrained(model_path, return_dict=True)

In [11]:
def generate(text,model,tokenizer):
    model.eval()
    input_ids = tokenizer.encode("WebNLG:{} </s>".format(text), return_tensors="pt")  
    outputs = model.generate(input_ids, max_length=300, temperature=0.1)
    return tokenizer.decode(outputs[0])

In [12]:
import unicodedata

In [14]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [13]:
import gensim

# Load Google's pre-trained Word2Vec model.
word2vecModel = gensim.models.KeyedVectors.load_word2vec_format('generation model/GoogleNews-vectors-negative300.bin', binary=True) 

In [14]:
#topics contient les sujets pour lesquels il y a des méthodes pour le dialogue
#quand on implémente des nouvelles méthodes il faut rajouter le sujet ici
topics=['music', 'job', 'food', 'game', 'sport', 'book', 'film', 'introduction']

In [15]:
nameofspeaker = ""
askedname = False
class Person:
   
    def __init__(self):
        self.age = random.randint(18, 80)
        self.name = unicodedata.normalize('NFD', random.choice(description[0])).encode('ascii', 'ignore').decode('utf-8')
        self.book = unicodedata.normalize('NFD', random.choice(description[1])).encode('ascii', 'ignore').decode('utf-8')
        self.sport = unicodedata.normalize('NFD', random.choice(description[2])).encode('ascii', 'ignore').decode('utf-8')
        self.music = unicodedata.normalize('NFD', random.choice(description[3])).encode('ascii', 'ignore').decode('utf-8')
        self.job = unicodedata.normalize('NFD', random.choice(description[4])).encode('ascii', 'ignore').decode('utf-8')
        self.film =  unicodedata.normalize('NFD', random.choice(description[5])).encode('ascii', 'ignore').decode('utf-8')
        self.game =  unicodedata.normalize('NFD', random.choice(description[6])).encode('ascii', 'ignore').decode('utf-8')
        self.food =  unicodedata.normalize('NFD', random.choice(description[7])).encode('ascii', 'ignore').decode('utf-8')
        self.country = unicodedata.normalize('NFD', random.choice(description[8])).encode('ascii', 'ignore').decode('utf-8')
        
        
    def generateDescription(self):
        surname=self.name.split(' ')[0]
        string=self.name+' | age | '+str(self.age)+' && '+self.name+' | country | '+self.country
        print(generate(string, model, tokenizer))
        string=surname+' | job | '+self.job
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteBook | '+self.book
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteSport | '+self.sport
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteMusic | '+self.music
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteFilm | '+self.film
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteGame | '+self.game
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteFood | '+self.food
        print(generate(string, model, tokenizer))

    def showProperties(self):
        print('name: '+self.name+'\nage: '+str(self.age)+'\ncountry: '+self.country+'\njob: '+self.job+'\nbook: '+self.book+'\nsport: '+self.sport+'\nmusic: '+self.music+'\nfilm: '+self.film+'\ngame: '+self.game+'\nfood: '+self.food+'\n')
        
    def introduce(self):
        text=self.name.split(' ')[0]+':   '

        #Greeting
        rand=random.randint(0,3)
        if rand==0:
            text+='Hello, '
        elif rand==1:
            text+='Hi, '
        elif rand==2:
            text+='Hey, '
        elif rand==3:
            text+='Good day, '
      
        #Name
        rand=random.randint(0,2)
        if rand==0:
            text+="I'm "
        elif rand==1:
            text+="I am "
        elif rand==2:
            text+="my name is "
        text+=self.name+', '

        #Age
        rand=random.randint(0,1)
        if rand==0:
            text+='I am '
        elif rand==1:
            text+="I'm "
        text+=str(self.age)+' years old '

        #Country
        rand=random.randint(0,3)
        if rand==0:
            text+="and I'm from "
        elif rand==1:
            text+="and I am from "
        elif rand==2:
            text+="and I come from "
        elif rand==3:
            text+="and I live in "
        text+=self.country+'.'

        print(text)

    def tellYourFavouriteFood(self):
        text=''
        rand=random.randint(0,4)
        if rand==0:
            text+='My favourite food is '
        elif rand==1:
            text+='I like to eat '
        elif rand==2:
            text+='I love '
        elif rand==3:
            text+='My favourite dish is '
        elif rand==4:
            text+='I really enjoy '
        text+=self.food+'.'
        return text

    def startTopicFood(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteFood()+' '
        #Ask the others about their favourite food
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite food?'
        elif rand==1:
            text+='What do you like to eat most?'
        elif rand==2:
            text+='What is your preferred dish?'
        elif rand==3:
            text+='What is your favourite dish?'
        elif rand==4:
            text+='Which food do you enjoy most?'
        print(name+text)
        return text

    def respondToFood(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteFood()
        print(text)

    def tellYourJob(self):
        text=''
        rand=random.randint(0,4)
        if rand==0:
            text+='I work as a '
        elif rand==1:
            text+='I am '
        elif rand==2:
            text+="I'm "
        elif rand==3:
            text+='My job is '
        elif rand==4:
            text+='I make a living from working as a '
        text+=self.optimizeJob(self.job)+'.'
        return text
    
    def optimizeJob(self, job):
        split=job.split(' ')
        for i in range(len(split)):
            if split[i]=='management':
                split[i]='manager'
            elif split[i]=='support':
                split[i]='supporter'
            elif split[i]=='therapy':
                split[i]='therapist'
            elif split[i]=='supervision':
                split[i]='supervisor'
            elif split[i]=='scouting':
                split[i]='scout'
        optimizedJob=''
        for word in split:
            optimizedJob+=word+' '
        optimizedJob=optimizedJob[:len(optimizedJob)-1]
        return optimizedJob

    def startTopicJob(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourJob()+' '
        #Ask the others about their job
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your job?'
        elif rand==1:
            text+='What are you doing for a living?'
        elif rand==2:
            text+='What are you working?'
        elif rand==3:
            text+='What do you do?'
        elif rand==4:
            text+='What do you work as?'
        print(name+text)
        return text

    def respondToJob(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourJob()
        print(text)

    def tellYourFavouriteMusic(self):
        text=''
        rand=random.randint(0,5)
        if rand==0:
            text+='My favourite music is '
        elif rand==1:
            text+='I like listening to '
        elif rand==2:
            text+='I like '
        elif rand==3:
            text+='My favourite music genre is '
        elif rand==4:
            text+='I really enjoy listening to '
        elif rand==5:
            text+='I usually listen to '
        text+=self.music+'.'
        return text

    def startTopicMusic(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteMusic()+' '
        #Ask the others about their favourite music
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite music?'
        elif rand==1:
            text+='What do you like to listen to?'
        elif rand==2:
            text+='What is your favourite music genre?'
        elif rand==3:
            text+='What music do you like to listen to?'
        elif rand==4:
            text+='Which type of music do you enjoy most?'
        print(name+text)
        return text

    def respondToMusic(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteMusic()
        print(text)
        
    def tellYourFavouriteGame(self):
        text=''
        rand=random.randint(0,5)
        if rand==0:
            text+='My favourite game is '
        elif rand==1:
            text+='I like playing '
        elif rand==2:
            text+='I like '
        elif rand==3:
            text+='I like to play '
        elif rand==4:
            text+='I really enjoy playing '
        elif rand==5:
            text+='I often play '
        text+=self.game+'.'
        return text

    def startTopicGame(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteGame()+' '
        #Ask the others about their favourite game
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite game?'
        elif rand==1:
            text+='What do you like to play?'
        elif rand==2:
            text+='Which games do you play?'
        elif rand==3:
            text+='Are there games that you like to play?'
        elif rand==4:
            text+='Which game do you enjoy most?'
        print(name+text)
        return text

    def respondToGame(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteGame()
        print(text)
        
    def tellYourFavouriteSport(self):
        text=''
        rand=random.randint(0,9)
        if rand==0:
            text+='My favourite sport is '
        elif rand==1:
            text+='I like playing '
        elif rand==2:
            text+='I like '
        elif rand==3:
            text+='I like to play '
        elif rand==4:
            text+='I really enjoy playing '
        elif rand==5:
            text+='I often play '
        elif rand==6:
            text+='I like watching '
        elif rand==7:
            text+='I like to watch '
        elif rand==8:
            text+='I\'m a fan of '
        elif rand==9:
            text+='I\'m a '+self.sport+' fan.'
        if not rand==9:
            text+=self.sport+'.'
        return text

    def startTopicSport(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteSport()+' '
        #Ask the others about their favourite sport
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite sport?'
        elif rand==1:
            text+='Are you a sports fan?'
        elif rand==2:
            text+='Which sport are you interested in?'
        elif rand==3:
            text+='What is your preferred sport?'
        elif rand==4:
            text+='Which kind of sport do you like most?'
        print(name+text)
        return text

    def respondToSport(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteSport()
        print(text)
        
    def tellYourFavouriteFilm(self):
        text=''
        rand=random.randint(0,4)
        if rand==0:
            text+='My favourite film is '
        elif rand==1:
            text+='I really liked the film '
        elif rand==2:
            text+='I really enjoyed watching the film '
        elif rand==3:
            text+='I like films like '
        text+=self.film
        if rand<4:
            text+='.'
        elif rand==4:
            text+=' was the film that I liked most.'
        return text
    
    def startTopicFilm(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteFilm()+' '
        rand=random.randint(0,3)
        if rand==0:
            text+='What is your favourite film?'
        elif rand==1:
            text+='Which film did you like most?'
        elif rand==2:
            text+='Which films do watch?'
        elif rand==3:
            text+='Which film did you enjoy most?'
        print(name+text)
        return text
    
    def respondToFilm(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteFilm()
        print(text)
        
    def tellYourFavouriteBook(self):
        text=''
        rand=random.randint(0,5)
        if rand==0:
            text+='My favourite book is '
        elif rand==1:
            text+='I really like the book '
        elif rand==2:
            text+='I enjoyed reading the book '
        elif rand==3:
            text+='I am currently reading '
        elif rand==4:
            text+='I like the book '
        elif rand==5:
            text+='I find '
        text+=self.book
        if rand<3:
            text+='.'
        elif rand==3:
            text+=' and i really like it.'
        elif rand==4:
            text+=' a lot.'
        elif rand==5:
            text+=' is a pretty good book.'
        return text
    
    def startTopicBook(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteBook()+' '
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite book?'
        elif rand==1:
            text+='What do you like to read?'
        elif rand==2:
            text+='Which kind of books do you like?'
        elif rand==3:
            text+='Do you read books?'
        elif rand==4:
            text+='Which book do you like most?'
        print(name+text)
        return text
    
    def respondToBook(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteBook()
        print(text)

    def respond(self, message):
        #TODO alles beachten
        global nameofspeaker
        phrases = []
        questions = []
        words = ""
        for letters in message:
            if(letters == '.' or letters == '!'):
                phrases.append(words)
                words = ""
            elif(letters == '?'):
                questions.append(words)
                words = ""
            else:    
                words+=letters     
        
        SWITCH_DICT={
            'sport':self.respondToSport,
            'music':self.respondToMusic,
            'job':self.respondToJob,
            'game':self.respondToGame,
            'food':self.respondToFood,
            'film':self.respondToFilm,
            'book':self.respondToBook,
        }
        topicsquestions = []
        if(len(questions)!= 0):
            for question in questions:
                guessedTopic=self.guessTopic(question)
                if not guessedTopic==None:
                    topicsquestions.append(guessedTopic)
        if(len(topicsquestions)!=0):            
            SWITCH_DICT.get(topicsquestions[random.randint(0,len(topicsquestions)-1)])()
        topicsphrases = []
        
        if(len(phrases) != 0):
            for phrase in phrases:
                guessedTopic=self.guessTopic(phrase)
                if not guessedTopic==None:
                    topicsphrases.append(guessedTopic)
        if(len(topicsphrases)!=0):            
            SWITCH_DICT.get(topicsphrases[random.randint(0,len(topicsphrases)-1)])()
        if(len(topicsquestions)==0 and len(topicsphrases)==0):     
            print("I'm sorry  " + nameofspeaker + ". I don't know what you are speaking about. :( \n Do you wan't to speak about sport, Films, music ?")
        
    def guessTopic(self, text):
        bestTopic=None
        bestDistance=None
        words=text.split(' ')
        for word in words:
            #print(word)
            for topic in topics:
                try:
                    distance=word2vecModel.distance(word, topic)
                    #print(distance)
                    if bestTopic==None or distance<bestDistance:
                        bestTopic=topic
                        bestDistance=distance
                except:
                    continue
        return bestTopic
    
    def startTopic(self, doneTopics):
        openTopics=[topic for topic in topics if topic not in doneTopics]
        if len(openTopics)==0:
            return None
        rand=random.randint(0,len(openTopics)-1)
        if openTopics[rand]=='music':
            doneTopics.append('music')
            return self.startTopicMusic()
        elif openTopics[rand]=='job':
            doneTopics.append('job')
            return self.startTopicJob()
        elif openTopics[rand]=='food':
            doneTopics.append('food')
            return self.startTopicFood()
        elif openTopics[rand]=='game':
            doneTopics.append('game')
            return self.startTopicGame()
        elif openTopics[rand]=='sport':
            doneTopics.append('sport')
            return self.startTopicSport()
        elif openTopics[rand]=='film':
            doneTopics.append('film')
            return self.startTopicFilm()
        elif openTopics[rand]=='book':
            doneTopics.append('book')
            return self.startTopicBook()
        else:
            return None

In [18]:
person = Person()
person.respond("my name is Leo . What sport do you like ? i love Sandwiches.")

Csaba:   I'm a water polo fan.
Csaba:   My favourite book is The Happiness Hypothesis.


In [17]:
def generatePersons(nPersons):
    persons=[]
    for i in range(nPersons):
        person = Person()
        persons.append(person)
    return persons

In [18]:
def startChat(persons, nTopics):
    doneTopics=[]
    for person in persons: 
        person.introduce()
    for i in range(nTopics):
        personToStart=persons[random.randint(0, len(persons)-1)]
        message=personToStart.startTopic(doneTopics)
        if message==None:
            return
        random.shuffle(persons)
        for person in persons:
            if not person==personToStart:
                person.respond(message)

In [19]:
import time
def startChatInteractive(persons):
    global nameofspeaker
    nameofspeaker=input('Input your name : ')
    for person in persons: 
        person.introduce()
        time.sleep(2)
    while True:
        message=input(nameofspeaker + ': ')
        if message=='bye':
            return
        if not message=='':
            random.shuffle(persons)
            for person in persons:
                time.sleep(2)
                res=person.respond(message)
                if res==-1:
                    break           

In [20]:
def generateDescriptions(persons):
    for person in persons:
        person.generateDescription()
        print('\n')

In [21]:
myPersons=generatePersons(4)

In [89]:
startChat(myPersons, 5)           

John:   Hello, I am John VI, I am 69 years old and I am from Albania.
Ivan:   Hey, I'm Ivan the Terrible, I am 23 years old and I live in Guatemala.
Kevin:   Good day, my name is Kevin Johnson, I am 48 years old and I come from Andorra.
Grand:   Hey, I'm Grand Duchess Maria Pavlovna of Russia, I'm 61 years old and I'm from Zambia.
John:   I like playing Jogo Quebra-Cabeca. Which game do you enjoy most?
Ivan:   I like Power Rangers Collectible Card Game.
Ivan:   I often play Power Rangers Collectible Card Game.
Kevin:   I really enjoy playing Fish out of Water.
Kevin:   My favourite game is Fish out of Water.
Grand:   I like to play Hyperborea.
Grand:   I like Hyperborea.
Kevin:   I'm a fan of wildwater canoeing. What is your favourite sport?
Ivan:   I like playing sumo.
Ivan:   I'm a sumo fan.
John:   I like mountain running.
John:   I really enjoy playing mountain running.
Grand:   My favourite sport is Rapid ball.
Grand:   I like watching Rapid ball.
Kevin:   I find The Divine Instit

In [22]:
startChatInteractive(myPersons)

Input your name : matthis
Jonathan:   Hello, I'm Jonathan Edwards, I am 36 years old and I live in Nicaragua.
Nikolaes:   Hey, my name is Nikolaes Heinsius, I'm 68 years old and I live in Libya.
Earl:   Hi, I am Earl Johnson, I'm 68 years old and I come from Slovenia.
David:   Hey, my name is David Gray, I am 32 years old and I live in Kiribati.
matthis: Hello, all. I wanted to ask what good films exist ?
David:   Hum Paanch was the film that I liked most.
David:   My job is supervising director.
Nikolaes:   My favourite film is Tomorrow, When the War Began.
Nikolaes:   I make a living from working as a Lord Marshal.
Earl:   My favourite film is Whiteout.
Earl:   My job is cupping therapist.
Jonathan:   Podaa Podi was the film that I liked most.
Jonathan:   I am aviation pioneer.
matthis: I found therapist really interresting what do you think about
I'm sorry  matthis. I don't know what you are speaking about. :( 
 Do you wan't to speak about sport, Films, music ?
I'm sorry  matthis. I

KeyboardInterrupt: Interrupted by user

In [ ]:
generateDescriptions(myPersons)

In [ ]:
for person in myPersons:
    person.showProperties()

In [ ]:
generate('La Strada | published | 1954 && La strada | director | Federico Fellini', model, tokenizer)

In [ ]:
generate('Meju | fermented soybean bricks', model, tokenizer)

In [ ]:
generate('lumpia Semarang | spring roll from Indonesia', model, tokenizer)

In [3]:
word2vecModel.distance('judo','nice')

0.9496366940438747

In [1]:
word2vecModel.most_similar(positive=['university'], topn = 3)

NameError: name 'word2vecModel' is not defined